# Workshop 2: Bayesian Filters

Before we start with Bayesian filters, let's make sure we are familiar with the basics of the probability theory and the Bayes' Rule.

## 1. Conditional Probabilities

Conditional probability is a measure of the probability of an event given that (by assumption, presumption, assertion or evidence) another event has occurred (source: <a href="https://en.wikipedia.org/wiki/Conditional_probability">Wikipedia</a>).

$$ P (A\,/\,B) = P (A\,\text{and}\,B)\;/\;P(B) = P(B\,/\,A) \cdot P(A)\;/\;P(B) $$

<div class="alert alert-block alert-success">
<b>Quiz: </b> The probability a person is 1.90 m tall is 0.05, the probability a person is Austrian and is 1.90 m tall is 0.01. What is the probability that a person is Austrian given that the person is 1.90 m tall?
</div>

<div class="alert alert-block alert-success">
<b>Quiz: </b> Initially the person can be anywhere. What is the probability that the person is in cell A, given that it hears WiFi A?

<img src="img/ws2/quiz2.png">
</div>



## 2. Bayesian Filters in One Slide (Discrete Case)

Imagine a robot resides in a one-dimensional world with no idea where it is in this world. Since our robot is completely clueless about its location, it believes that every point in this one dimensional world is equally likely to be its current position. We can describe this mathematically by saying that the robot's probability function is uniform (the same) over the sample space. This represents the state of maximum confusion.

<img src="img/ws2/bayesian_filter_basics.png">

Assume there are three landmarks, which are three red doors that all look alike and we can distinguish a door from a non-door area (blue). Since the robot has just sensed that it is near a door, it assigns these locations greater probability (indicated by the bumps in the graph) whereas, all of the other locations have decreased belief. This function represents another probability distribution, called the posterior belief where the function is defined after the robot's sense measurement has been taken. The posterior function is the best representation of the robot's current belief, where each bump represents the robot's evaluation of its position relative to a door.

If the robot moves to the right a certain distance, we can shift the belief according to the motion. Notice that all of the bumps also shift to the right, as we would expect. What may come as a surprise, however, is that these bumps have not shifted perfectly. They have also flattened. This flattening is due to the uncertainty in robot motion: since the robot doesn't know exactly how far it has moved, its knowledge has become less precise and so the bumps have become less sharp. 

When we shift and flatten these bumps, we are performing a convolution. Convolution is a mathematical operation that takes two functions and measures their overlap. To be more specific, it measures the amount of overlap as you slide one function over another. For example, if two functions have zero overlap, the value of their convolution will be equal to zero. If they overlap completely, their convolution will be equal to one. As we slide between these two extreme cases, the convolution will take on values between zero and one.

In our convolution, the first function is the belief function (labeled "posterior" above), and the second is the function which describes the distance moved, which we will address in more depth later. The result of this convolution is the shifted and flattened belief function shown below. Now, assume that after the robot moves it senses itself right next to a door again so that the measurement is the same as before. Just like after our first measurement, the sensing of a door will increase our probability function by a certain factor everywhere where there is a door. So, should we get the same posterior belief that we had after our first measurement? No! Because unlike with our first measurement, when we were in a state of maximum uncertainty, this time we have some idea of our location prior to sensing. This prior information, together with the second sensing of a door, combine to give us a new probability distribution, as shown in the bottom graph.

In this graph we see a few minor bumps, but only one sharp peak. This peak corresponds to the second door. We have already explained this mathematically, but let's think intuitively about what happened. First, we saw the first door. This led us to believe that we were near some door, but we didn't know which. Then we moved and saw another door. We saw two doors in a row! So of course our probability function should have a major peak near the only location where we would expect to see two doors in a row.

Let's generalize our sense-and-move model. Below, $X$ represents location and $Z$ measurements:

<img src="img/ws2/bayesian_filter_basics2.png">


## Another Example

### Initial Belief

A robot __has a map__ of the grid below, but it doesn’t know where it is. 
<img src="img/ws2/bayesian_filter_step_1.png">

<div class="alert alert-block alert-success">
<b>Exercise: </b> What is the initial belief?
</div>


### Sensing
First let’s look at the sensing part:
<img src="img/ws2/bayesian_filter_step_0.png">

<img src="img/ws2/bayesian_filter_step_2.png">

<div class="alert alert-block alert-success">
<b>Exercise: </b> The observation z is <b style="color:red">red</b>, what is the new belief?
</div>

<img src="img/ws2/bayesian_filter_step_4.png">

### Movement

When using RF, you don’t want to see people teletransporting in your localization app. Movements models (any model) help a ton!

<img src="img/ws2/bayesian_filter_step_5.png">

<div class="alert alert-block alert-success">
<b>Exercise: </b> the robot is moving right, what is the new distribution?  
</div>
<img src="img/ws2/bayesian_filter_step_6.png">


<img src="img/ws2/bayesian_filter_step_7.png">

Congratulations you can now do discrete localization! Let's see how this can be used with phones!

## 3. Cookbook

### Step 1: Get RSSI signal for each cell

You have a map split into cells (the big room has two cells).
<img src="img/ws2/bayesian_cookbook_map.png">

### Step 2: Process signal and get histogram

Walk slowly inside each cell to gather RSS data. Collect different angles!

<img src="img/ws2/bayesian_cookbook_1.png" width="600">

Each cell has now a Probability Mass Function (PMF). The higher the difference between those in different cells the better!

### Step 3: Store data for offline analysis

<img src="img/ws2/bayesian_cookbook_2.png" width="600">

Localization won't be trained on the spot:
* Offline processing can help understand a method
* Gather data 3 minutes per cell
* Divide samples at random in test and training sets
* Build Bayesian Filter in your language of choice (I suggest to use Python or Matlab), train it with training set and check accuracy with testing set.
* Gather data at different days / times (multipath fading)

We have the radio map in the phone -- now, let's localize!

### Step 4: Get testing data

* Start with initial belief
* Do WiFi scan
* Sort access points in decreasing order, based on RSSI
* Choose highest, then second highest, ...

<img src="img/ws2/bayesian_cookbook_3.png">

### Step 5: Apply Bayes

Probability I am in cell $i$ given that I got RSSI measurement $r$ from access point $j$:

<img src="img/ws2/bayesian_cookbook_4.png">

### Step 6: When to stop iterations?

* No clear answer
* At every step you can update prior with
    * Data from other access points
    * New scans
* Stop when you
    * Pass a threshold ($\geq$ 95%)
    * Reach a steady state (oscillation around a certain probability)


### Step 7: Motion model
* Differentiate between idle and walking (we won't run :-) )
* Count number of steps $s$, or simply moving time $t$
    * Autocorrelation / Fourier transform can be used for counting steps
* For stride / time length assume uniform distribution
    * For example between 0.5 and 1.2 meters
* Hense given location $x$ the new location distribution is $[x-1.2m, x-0.5m] \cup [x+0.5m, x+1.2m]$
* We will only test on the map given above.


### Step 8: GUI

<div class="alert alert-block alert-info">
<b>Remember</b>: No training on the spot! We will push "Locate me" until the app converges.
</div>

<img src="img/ws2/bayesian_cookbook_gui.png" width="600">

## 4. Tips and Tricks

### Modeling radio maps
* We suggest to use tables to store radio maps
* But they have several limitations:
    * Sampling granularity: What if you get RSSI value that is not present?
    * Memory space: If you have N access points: 255 x C x N
    
<img src="img/ws2/bayesian_tips_1.png" width="600">

If you approximate your RSS data with a Gaussian distribution:
* Less memory space: 2 x C x N
* More granularity: $f(x, \mu, \sigma) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}$
* Better accuracy with low amount of observations
* Small scale fading can still bring surprising results but we will use several attempts when testing.

<img src="img/ws2/bayesian_tips_2.png" width="600">

### Modelling variance 
Check your testing phase on different days:
* Multipath fading occurs due to small changes
* People moving, opening / closing doors, ...

<img src="img/ws2/bayesian_tips_3.png" width="400">

### Concrete Example Guidelines
Published in scientific literature:
* Horus (Bayesian Grid)
    * 100 samples per cell spaced 300ms, cell size: 1.5m or 2.0m
    * Error: 2 meters
    * Paper: <a href="http://www.cs.umd.edu/%7Emoustafa/papers/horus_usenix.pdf"> The Horus WLAN Location Determination System</a>
* Practical Robust Localization (Bayesian room)
    * 60 seconds sampling per office, cell size: 2.5 x 5.0 meters (500 offices)
    * 95% accuracy with 2 or 3 RSS measurements
    * Paper: <a href="http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.80.6957&rep=rep1&type=pdf">Practical Robust Localization Over Large-Scale 802.11 Wireless Networks</a>
* Both methods provide few meters error for 1-minute sampling per cell
* WiFi scans take a few seconds
    * Dual band phones may bring new results. Extra: compare 2.4 and 5 GHZ bands?

### Diminishing Returns 
More training and testing data does not necessarily mean much better results, but may mean more work, time and use of resources (memory, processor). Work hard but smart in your localization app!

<img src="img/ws2/bayesian_tips_4.png" width="600">


***
# Credits
* Marco Zuniga's <a href="http://studiegids.tudelft.nl/a101_displayCourse.do?course_id=40368">"Smart Phone Sensing" Course at TU Delft</a>
* Parts of this workshop are based on content from: <a href="https://www.udacity.com/course/cs373">Artificial Intelligence for Robotics</a>